In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
df_train = pd.read_pickle('./data/df_train_vec.pkl')
df_test = pd.read_pickle('./data/df_test_vec.pkl')